# Chapter 6: Step-by-Step Reasoning

- [Lesson](#lesson)
- [Exercises](#exercises)
- [Example Playground](#example-playground)

## Setup

Run the following setup cell to load your API key and establish the `get_completion` helper function.

In [ ]:
!pip install openai python-dotenv

# Import python's built-in regular expression library
import re
from openai import OpenAI
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Access variables
API_KEY = os.getenv("API_KEY")
BASE_URL = "https://api.deepseek.com"
MODEL_NAME = "deepseek-chat"  # Note: Using the normal model for forcing the reasoning via prompting

# Store the variables for use across notebooks
%store API_KEY
%store BASE_URL
%store MODEL_NAME

client = OpenAI(
    api_key=API_KEY,
    base_url=BASE_URL
)

def get_completion(prompt: str, system_prompt="", prefill=""):
    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    
    messages.append({"role": "user", "content": prompt})
    
    # Only add assistant message if prefill is not empty
    if prefill:
        messages.append({"role": "assistant", "content": prefill})
    
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=messages,
        max_tokens=2000,
        temperature=0.0,
        stream=False
    )
    
    # For the reasoner model, we have both reasoning content and final answer
    # If reasoning_content exists, prepend it to the final content
    final_content = response.choices[0].message.content
    if hasattr(response.choices[0].message, 'reasoning_content') and response.choices[0].message.reasoning_content:
        final_content = f"<reasoning>{response.choices[0].message.reasoning_content}</reasoning>\n\n{final_content}"
    
    return final_content

---

## Lesson

If someone woke you up and immediately started asking you several complicated questions that you had to respond to right away, how would you do? Probably not as good as if you were given time to **think through your answer first**. 

Guess what? Language models are the same way.

**Giving a model time to think step by step sometimes makes it more accurate**, particularly for complex tasks. However, **thinking only counts when it's made explicit**. You cannot ask a model to think but output only the answer - in this case, no thinking has actually occurred.

### Examples

In the prompt below, it's clear to a human reader that the second sentence contradicts the first. But **the model might take the word "unrelated" too literally** and not realize the sarcasm.

In [4]:
# Prompt
PROMPT = """Is this movie review sentiment positive or negative?

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since the year 1900."""

# Print the model's response
print(get_completion(PROMPT))

The sentiment of this movie review is **positive**. The reviewer expresses enthusiasm and admiration for the movie's "freshness and originality," even though they humorously exaggerate their lack of exposure to modern films. The overall tone is appreciative and complimentary.


To improve the response, let's **allow the model to think things out first before answering**. We do that by literally spelling out the steps that the model should take in order to process and think through its task. Along with a dash of role prompting, this empowers the model to understand the review more deeply.

Note: DeepSeek's "reasoner" model is specifically designed for this type of step-by-step thinking, so we've set it as the default model for this chapter.

Note n2: Disabled for using the prompt engineering to force the model into thinking. If you want to use DeepSeek-R1, change MODEL_NAME to 'deepseek-reasoner'

In [6]:
# System prompt
SYSTEM_PROMPT = "You are a savvy reader of movie reviews."

# Prompt
PROMPT = """Is this review sentiment positive or negative? First, write the best arguments for each side in <positive-argument> and <negative-argument> XML tags, then answer.

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since 1900."""

# Print the model's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

<positive-argument>  
The review states that the movie "blew my mind with its freshness and originality," which is a strong positive statement. This suggests that the reviewer found the film to be innovative and highly enjoyable, which clearly indicates a positive sentiment.  
</positive-argument>  

<negative-argument>  
The reviewer sarcastically adds, "In totally unrelated news, I have been living under a rock since 1900," which implies that the movie's "freshness and originality" might not actually be groundbreaking or new. This sarcasm undermines the initial praise, suggesting that the reviewer found the film unoriginal or outdated, which points to a negative sentiment.  
</negative-argument>  

The sentiment of the review is **negative**. The sarcastic remark strongly undercuts the initial positive statement, indicating that the reviewer is being critical of the movie.



**Models can sometimes be sensitive to ordering**. This example is challenging for the model to understand nuanced text, and when we swap the order of the arguments from the previous example so that negative is first and positive is second, this might change the model's overall assessment.

In some situations, **the model might be more likely to choose the second of two options**, possibly because in its training data, second options were more likely to be correct. (I just tried, suprisingly true)

In [18]:
# Prompt
PROMPT = """Is this review sentiment negative or positive? First write the best arguments for each side in <negative-argument> and <positive-argument> XML tags, then answer.

This movie blew my mind with its freshness and originality. Unrelatedly, I have been living under a rock since 1900."""

# Print the model's response
print(get_completion(PROMPT))

<negative-argument>  
The review could be interpreted as negative because the second sentence, "Unrelatedly, I have been living under a rock since 1900," suggests sarcasm or irony. It implies that the reviewer might not actually find the movie fresh or original, but rather outdated or unimpressive, given their exaggerated claim of being out of touch for over a century.  
</negative-argument>  

<positive-argument>  
The review could be interpreted as positive because the first sentence, "This movie blew my mind with its freshness and originality," is a clear and enthusiastic compliment. The second sentence, while seemingly unrelated, might simply be a humorous aside and not intended to undermine the positive sentiment of the first sentence.  
</positive-argument>  

**Answer:** The review is likely **positive**. The first sentence is a strong, direct compliment, and the second sentence appears to be a humorous exaggeration rather than a critique of the movie.


**Letting the model think can shift its answer from incorrect to correct**. It's that simple in many cases where the model makes mistakes!

Let's go through an example where the model's answer might be incorrect to see how asking it to think can fix that.

In [8]:
# Prompt
PROMPT = "Name a famous movie starring an actor who was born in the year 1956."

# Print the model's response
print(get_completion(PROMPT))

One famous movie starring an actor born in 1956 is **"The Terminator"** (1984), featuring **Arnold Schwarzenegger**, who was born on July 30, 1956. This iconic film helped solidify Schwarzenegger's status as a major action star.


Let's fix this by asking the model to think step by step, this time in `<brainstorm>` tags.

In [19]:
# Prompt
PROMPT = "Name a famous movie starring an actor who was born in the year 1956. First brainstorm about some actors and their birth years in <brainstorm> tags, then give your answer."

# Print the model's response
print(get_completion(PROMPT))

<brainstorm>  
- Tom Hanks (born 1956)  
- Mel Gibson (born 1956)  
- Carrie Fisher (born 1956)  
- Bryan Cranston (born 1956)  
- Tim Curry (born 1956)  
</brainstorm>  

One famous movie starring an actor born in 1956 is *Forrest Gump*, starring Tom Hanks.


If you would like to experiment with the lesson prompts without changing any content above, scroll all the way to the bottom of the lesson notebook to visit the [**Example Playground**](#example-playground).

---

## Exercises
- [Exercise 6.1 - Classifying Emails](#exercise-61---classifying-emails)
- [Exercise 6.2 - Email Classification Formatting](#exercise-62---email-classification-formatting)

### Exercise 6.1 - Classifying Emails
In this exercise, we'll be instructing the model to sort emails into the following categories:										
- (A) Pre-sale question
- (B) Broken or defective item
- (C) Billing question
- (D) Other (please explain)

For the first part of the exercise, change the `PROMPT` to **make the model output the correct classification and ONLY the classification**. Your answer needs to **include the letter (A - D) of the correct choice, with the parentheses, as well as the name of the category**.

Refer to the comments beside each email in the `EMAILS` list to know which category that email should be classified under.

(I managed to solve this by just changing the order of the prompt shceme. I brought the categories down, and the "SOMETIMES there an be MORE than one answer" at last. This way, the model's last thing in his mind is that there can be more than one thing and outputs the correct classificationi for the second email: A and D)

In [20]:
# Prompt template with a placeholder for the variable content
PROMPT = """
         Please classify this email as one of the following categories:

         <email>{email}</email>
         Your answer needs to **include the letter (A - D) of the correct choice, with the parentheses, as well as the name of the category**

         You can **ONLY** output the classification/s.
         {{
         "categories": {{
                        "(A)": "Pre-sale question",
                        "(B)": "Broken or defective item",
                        "(C)": "Billing question",
                        "(D)": "Other (please explain)"
                        }}
         }}
         SOMETIMES there can be MORE than one category for each.
         """

# Prefill for the model's response, if any
PREFILL = ""

# Variable content stored as a list
EMAILS = [
    "Hi -- My Mixmaster4000 is producing a strange noise when I operate it. It also smells a bit smoky and plasticky, like burning electronics.  I need a replacement.", # (B) Broken or defective item
    "Can I use my Mixmaster 4000 to mix paint, or is it only meant for mixing food?", # (A) Pre-sale question OR (D) Other (please explain)
    "I HAVE BEEN WAITING 4 MONTHS FOR MY MONTHLY CHARGES TO END AFTER CANCELLING!!  WTF IS GOING ON???", # (C) Billing question
    "How did I get here I am not good with computer.  Halp." # (D) Other (please explain)
]

# Correct categorizations stored as a list of lists to accommodate the possibility of multiple correct categorizations per email
ANSWERS = [
    ["B"],
    ["A","D"],
    ["C"],
    ["D"]
]

# Dictionary of string values for each category to be used for regex grading
REGEX_CATEGORIES = {
    "A": "A() P",
    "B": "B() B",
    "C": "C() B",
    "D": "D() O"
}

# Iterate through list of emails
for i,email in enumerate(EMAILS):
    
    # Substitute the email text into the email placeholder variable
    formatted_prompt = PROMPT.format(email=email)
   
    # Get the model's response
    response = get_completion(formatted_prompt, prefill=PREFILL)

    # Grade the model's response
    grade = any([bool(re.search(REGEX_CATEGORIES[ans], response)) for ans in ANSWERS[i]])
    
    # Print the model's response
    print("--------------------------- Full prompt with variable substutions ---------------------------")
    print("USER TURN")
    print(formatted_prompt)
    print("\nASSISTANT TURN")
    print(PREFILL)
    print("\n------------------------------------- Model's response -------------------------------------")
    print(response)
    print("\n------------------------------------------ GRADING ------------------------------------------")
    print("This exercise has been correctly solved:", grade, "\n\n\n\n\n\n")

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN

         Please classify this email as one of the following categories:

         <email>Hi -- My Mixmaster4000 is producing a strange noise when I operate it. It also smells a bit smoky and plasticky, like burning electronics.  I need a replacement.</email>
         Your answer needs to **include the letter (A - D) of the correct choice, with the parentheses, as well as the name of the category**

         You can **ONLY** output the classification/s.
        {
         "categories": {
                        "(A)": "Pre-sale question",
                        "(B)": "Broken or defective item",
                        "(C)": "Billing question",
                        "(D)": "Other (please explain)"
                        }
         }
        SOMETIMES there can be MORE than one category for each.
         

ASSISTANT TURN


------------------------------------- Model's response -

❓ If you want a hint, run the cell below!

In [ ]:
from hints import exercise_6_1_hint; print(exercise_6_1_hint)

Still stuck? Run the cell below for an example solution.						

In [ ]:
from hints import exercise_6_1_solution; print(exercise_6_1_solution)

### Exercise 6.2 - Email Classification Formatting
In this exercise, we're going to refine the output of the above prompt to yield an answer formatted exactly how we want it. 

Use your favorite output formatting technique to make the model wrap JUST the letter of the correct classification in `<answer></answer>` tags. For instance, the answer to the first email should contain the exact string `<answer>B</answer>`.

Refer to the comments beside each email in the `EMAILS` list if you forget which letter category is correct for each email.

(This one has been the hardest until now. I had to require Grok's help to make this prompt)

In [33]:
# Prompt template with a placeholder for the variable content
PROMPT = """
            Please classify this email into one or more of the following categories:

            <email>{email}</email>

            Your answer needs to **include the letter (A - D) of each correct choice, enclosed in separate <answer></answer> tags**.  
            If the email fits into more than one category, include all applicable letters as shown in the example below.  
            You can **ONLY** output the `<answer>` tags with the classifications.

            {{
            "categories": {{
                "(A)": "Pre-sale question",
                "(B)": "Broken or defective item",
                "(C)": "Billing question",
                "(D)": "Other (please explain)"
            }}
            }}

            **Example of multiple classifications:**  
            For an email like: "My Mixmaster4000 is broken, and I was charged twice for it. Please help."  
            You would output:  
            <answer>B</answer>  
            <answer>C</answer>
         """

# Prefill for the model's response, if any
PREFILL = ""

# Variable content stored as a list
EMAILS = [
    "Hi -- My Mixmaster4000 is producing a strange noise when I operate it. It also smells a bit smoky and plasticky, like burning electronics.  I need a replacement.", # (B) Broken or defective item
    "Can I use my Mixmaster 4000 to mix paint, or is it only meant for mixing food?", # (A) Pre-sale question OR (D) Other (please explain)
    "I HAVE BEEN WAITING 4 MONTHS FOR MY MONTHLY CHARGES TO END AFTER CANCELLING!!  WTF IS GOING ON???", # (C) Billing question
    "How did I get here I am not good with computer.  Halp." # (D) Other (please explain)
]

# Correct categorizations stored as a list of lists to accommodate the possibility of multiple correct categorizations per email
ANSWERS = [
    ["B"],
    ["A","D"],
    ["C"],
    ["D"]
]

# Dictionary of string values for each category to be used for regex grading
REGEX_CATEGORIES = {
    "A": "<answer>A</answer>",
    "B": "<answer>B</answer>",
    "C": "<answer>C</answer>",
    "D": "<answer>D</answer>"
}

# Iterate through list of emails
for i,email in enumerate(EMAILS):
    
    # Substitute the email text into the email placeholder variable
    formatted_prompt = PROMPT.format(email=email)
   
    # Get the model's response
    response = get_completion(formatted_prompt, prefill=PREFILL)

    # Grade the model's response
    grade = any([bool(re.search(REGEX_CATEGORIES[ans], response)) for ans in ANSWERS[i]])
    
    # Print the model's response
    print("--------------------------- Full prompt with variable substutions ---------------------------")
    print("USER TURN")
    print(PREFILL)
    print("\n------------------------------------- Model's response -------------------------------------")
    print(response)
    print("\n------------------------------------------ GRADING ------------------------------------------")
    print("This exercise has been correctly solved:", grade, "\n\n\n\n\n\n")

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN


------------------------------------- Model's response -------------------------------------
<answer>B</answer>

------------------------------------------ GRADING ------------------------------------------
This exercise has been correctly solved: True 






--------------------------- Full prompt with variable substutions ---------------------------
USER TURN


------------------------------------- Model's response -------------------------------------
<answer>A</answer>  
<answer>D</answer>

------------------------------------------ GRADING ------------------------------------------
This exercise has been correctly solved: True 






--------------------------- Full prompt with variable substutions ---------------------------
USER TURN


------------------------------------- Model's response -------------------------------------
<answer>C</answer>

----------------------------

❓ If you want a hint, run the cell below!

In [28]:
from hints import exercise_6_2_hint; print(exercise_6_2_hint)

The grading function in this exercise is looking for only the correct letter wrapped in <answer> tags, such as "<answer>B</answer>". The correct categorization letters are the same as in the above exercise.
Sometimes the simplest way to go about this is to give Claude an example of how you want its output to look. Just don't forget to wrap your example in <example></example> tags! And don't forget that if you prefill Claude's response with anything, Claude won't actually output that as part of its response.


### Congrats!

If you've solved all exercises up until this point, you're ready to move to the next chapter. Happy prompting!

---

## Example Playground

This is an area for you to experiment freely with the prompt examples shown in this lesson and tweak prompts to see how it may affect the model's responses.

In [ ]:
# Prompt
PROMPT = """Is this movie review sentiment positive or negative?

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since the year 1900."""

# Print the model's response
print(get_completion(PROMPT))

In [ ]:
# System prompt
SYSTEM_PROMPT = "You are a savvy reader of movie reviews."

# Prompt
PROMPT = """Is this review sentiment positive or negative? First, write the best arguments for each side in <positive-argument> and <negative-argument> XML tags, then answer.

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since 1900."""

# Print the model's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

In [ ]:
# Prompt
PROMPT = """Is this review sentiment negative or positive? First write the best arguments for each side in <negative-argument> and <positive-argument> XML tags, then answer.

This movie blew my mind with its freshness and originality. Unrelatedly, I have been living under a rock since 1900."""

# Print the model's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "Name a famous movie starring an actor who was born in the year 1956."

# Print the model's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "Name a famous movie starring an actor who was born in the year 1956. First brainstorm about some actors and their birth years in <brainstorm> tags, then give your answer."

# Print the model's response
print(get_completion(PROMPT))